In [2]:
import h5py
import pandas as pd
import numpy as np

DATA_DIR = "../../data/logs"

In [3]:
with h5py.File(DATA_DIR + '/preprocessed.h5','r') as hf:
    nextImages = hf['nextImage'][...]
    currentImages = hf['currentImage'][...]
    metadata = hf['metadata'][...]
    ids = hf['ids'][...]

In [8]:
df = pd.DataFrame([{ 
    "currentImage": ci,
    "nextImage": ni,
    "action": int(md[0]),
    "reward": md[1],
    "died": int(md[2]),
    "didBoost": int(md[3]),
    "id": i.decode()} for ni, ci, md, i in zip(nextImages, currentImages, metadata, ids)])

In [11]:
df = df.loc[df['id'] != "00000000-0000-0000-0000-000000000000"]

In [13]:
def min_max(x):
    _min = np.min(x)
    _max = np.max(x)

    return (x - _min) / (_max - _min) * 2.0 - 1.0

5238     100.0
5239     100.0
5240       1.0
5241       1.0
5242       1.0
         ...  
94189      2.0
94190      0.0
94191      0.0
94192      0.0
94193     -1.0
Name: reward, Length: 83718, dtype: float64